# Stationäre Antwort für Balken mit Tilger

[Aufgabenstellung](calculations/MMS_%C3%9Cbung%203_Tilger.pdf)

![Statisches System des Balkens mit Tilger](pictures/mms4system.jpg){#fig-system}

In [1]:
import sympy as sp 
from sympycalcs import render, convert
import sympy.physics.units as unit

import matplotlib.pyplot as plt

## Parameter der Aufgabenstellung

Die Auslegung des Tilgers kann folgender massen geschehen:

- Tilgermasse $5\%$ von der Masse des Hauptträgers.
- Optimale Frequenz bestimmen: $$f_{T,opt} =\frac{f_H}{1+\frac{m_T}{m_H}}$$
- Daraus die optimale Steifigkeit bestimmen:$$k_{T,opt} = (2 \pi f_{T,opt})^2$$

In [2]:
params = {'E': 200*10**3 *unit.N/unit.mm**2,
          'I':2*10**8*unit.mm**4,
          'm_H':2000*unit.N*unit.second**2/unit.m,
          'm_T':100*unit.N*unit.second**2/unit.m,
          'k_T':90*10**3*unit.N/unit.m,
          'l':5*unit.m,
          'F_0':0.8*10**3*unit.N,
          'omega':12.6/unit.second,
          'phi_11':1,
          'phi_12':1,
          'zeta':0.0         
          }

render.dict_render(params)

Eq(E, 200000*newton/millimeter**2)

Eq(I, 200000000*millimeter**4)

Eq(m_H, 2000*newton*second**2/meter)

Eq(m_T, 100*newton*second**2/meter)

Eq(k_T, 90000*newton/meter)

...

Eq(F_0, 800.0*newton)

Eq(omega, 12.6/second)

Eq(phi_11, 1)

Eq(phi_12, 1)

Eq(zeta, 0.0)

In [3]:
E, I, m_H, m_T, k_T, l, F_0, omega, delta_11, delta_12 = sp.symbols('E, I, m_H, m_T, k_T, l, F_0, omega, delta_11, delta_12')

t = sp.symbols('t')

In [4]:
F_t = F_0 * sp.sin(omega*t)

render.eq_display('F(t)', F_t,
                  'F(t)', F_t.subs(params))

Eq(F(t), F_0*sin(omega*t))

Eq(F(t), 800.0*newton*sin(12.6*t/second))

## Steifigkeitsmatrix $\mathbf{K}$


![Verformungen an den beiden Freiheitsgraden](pictures/mms3_steifigkeit.jpg){#fig-verformungen}


Wichtig dabei sind die Richtungen der Kräfte. Als Denkstütze gilt folgendes:
- Der Auslenkung um $u$ wirkt die Federkraft entgegen, welche $k u$ entspricht.
- Zusätzlich wirkt die Trägheitskraft der Auslenkung entgegen, welche $m u''$ entspricht.
- Nach der Betrachtung des ausgelenkten Punkts, kann mittels *Actio-Reactio*-Prinzip das "*Stockwerk*" ins Gleichgewicht gebracht werden.
- Vorzeichen sind gegen der Bewegungsrichtig positiv.

In [5]:
k_T, k_H = sp.symbols('k_T, k_H')

params['k_H'] = (48 * E*I / (2*l)**3).subs(params).simplify()
K = sp.Matrix([[k_H + k_T, -k_T],[-k_T, k_T]])


render.eq_display(sp.MatrixSymbol('K', 2,2), K,
                  sp.MatrixSymbol('K', 2,2), K.subs(params),
                  )

Eq(K, Matrix([
[k_H + k_T, -k_T],
[     -k_T,  k_T]]))

Eq(K, Matrix([
[2010000*newton/meter, -90000*newton/meter],
[ -90000*newton/meter,  90000*newton/meter]]))

## Eigenvektoren
### Massenmatrix $\mathbf{M}$ 

Die Massenmatrix folgt dem gleichen Aufbau wie die Steifigkeitsmatrix. Es gelten die gleichen Vorzeichenregelungen. Die Einträge beziehen sich auf @fig-verformungen .

In [6]:
M = sp.Matrix([[m_H, 0],[0, m_T]])

render.eq_display(sp.MatrixSymbol('M', 2,2), M,
                  sp.MatrixSymbol('M', 2,2), M.subs(params))

Eq(M, Matrix([
[m_H,   0],
[  0, m_T]]))

Eq(M, Matrix([
[2000*newton*second**2/meter,                          0],
[                          0, 100*newton*second**2/meter]]))

### Eigenkreisfrequenzen 
Bei einem Mehrmassenschwinger gibt es entsprechend den Freiheitsgraden Eigenkreisfrequenzen $\omega_n$. Diese lassen sich anhand folgender Gleichung bestimmen:

$$\det{[\mathbf{K}-\omega_n^2 \mathbf{M}]=0}$$

In [7]:
omega_n =sp.symbols('omega_n')
eq_omega = sp.det(K-omega_n**2*M)

omega_n_solve = sp.solve(eq_omega, omega_n)
omega_1 = omega_n_solve[1]
omega_2 = omega_n_solve[3]

render.eq_display('omega_1', omega_1.subs(params).simplify().evalf(3),
                  'omega_2', omega_2.subs(params).simplify().evalf(3))

Eq(omega_1, 27.3/second)

Eq(omega_2, 34.1/second)

### Eigenvektoren $\mathbf{\phi}$

Dazu ist die entsprechende Normierung aus der Aufgabenstellung zu berücksichtigen.

In [8]:
phi_11, phi_21, phi_12, phi_22 = sp.symbols('phi_11, phi_21, phi_12, phi_22')

phi_1 = sp.Matrix([[phi_11], [phi_21]])

phi_21 = list(sp.solve((K-omega_1**2 *M)*phi_1, phi_21).values())[0]

params['phi_21'] = phi_21.subs(params).simplify() 




render.eq_display((K-omega_1**2 *M)*phi_1,sp.Matrix([[0],[0]]),
                  sp.MatrixSymbol('phi_1', 2,1),sp.simplify(phi_1.subs(params)).evalf(3))


Eq(Matrix([
[-k_T*phi_21 + phi_11*(k_H + k_T - m_H*(k_H/(2*m_H) + k_T/(2*m_T) + k_T/(2*m_H) - sqrt(k_H**2*m_T**2 - 2*k_H*k_T*m_H*m_T + 2*k_H*k_T*m_T**2 + k_T**2*m_H**2 + 2*k_T**2*m_H*m_T + k_T**2*m_T**2)/(2*m_H*m_T)))],
[      -k_T*phi_11 + phi_21*(k_T - m_T*(k_H/(2*m_H) + k_T/(2*m_T) + k_T/(2*m_H) - sqrt(k_H**2*m_T**2 - 2*k_H*k_T*m_H*m_T + 2*k_H*k_T*m_T**2 + k_T**2*m_H**2 + 2*k_T**2*m_H*m_T + k_T**2*m_T**2)/(2*m_H*m_T)))]]), Matrix([
[0],
[0]]))

Eq(phi_1, Matrix([
[ 1.0],
[5.79]]))

In [9]:
phi_2 = sp.Matrix([[phi_12], [phi_22]])

phi_22 = list(sp.solve((K-omega_2**2 *M)*phi_2, phi_22).values())[0]

params['phi_22'] = phi_22.subs(params).simplify() 




render.eq_display((K-omega_2**2 *M)*phi_2,sp.Matrix([[0],[0]]),
                  sp.MatrixSymbol('phi_2', 2,1),sp.simplify(phi_2.subs(params)).evalf(3))


Eq(Matrix([
[-k_T*phi_22 + phi_12*(k_H + k_T - m_H*(k_H/(2*m_H) + k_T/(2*m_T) + k_T/(2*m_H) + sqrt(k_H**2*m_T**2 - 2*k_H*k_T*m_H*m_T + 2*k_H*k_T*m_T**2 + k_T**2*m_H**2 + 2*k_T**2*m_H*m_T + k_T**2*m_T**2)/(2*m_H*m_T)))],
[      -k_T*phi_12 + phi_22*(k_T - m_T*(k_H/(2*m_H) + k_T/(2*m_T) + k_T/(2*m_H) + sqrt(k_H**2*m_T**2 - 2*k_H*k_T*m_H*m_T + 2*k_H*k_T*m_T**2 + k_T**2*m_H**2 + 2*k_T**2*m_H*m_T + k_T**2*m_T**2)/(2*m_H*m_T)))]]), Matrix([
[0],
[0]]))

Eq(phi_2, Matrix([
[  1.0],
[-3.46]]))

### Orthogonalitätsbedingung

Zur effektiven Entkoppelung der Gleichungen muss die Orthogonalitätsbedingung eingehalten sein. Dies gilt es für die Massenmatrix zu kontrollieren:

$$\mathbf{\phi_1^T M \phi_1 \neq 0}$$

$$\mathbf{\phi_2^T M \phi_2 \neq 0}$$

$$\mathbf{\phi_2^T M \phi_1 = 0}$$

Sowohl auch für die Steifigkeitsmatrix:

$$\mathbf{\phi_1^T K \phi_1 \neq 0}$$

$$\mathbf{\phi_2^T K \phi_2 \neq 0}$$

$$\mathbf{\phi_2^T K \phi_1 = 0}$$



In [10]:
render.eq_display(sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_1',2,1),(phi_1.T*M*phi_1).subs(params).evalf(3),
                  
                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_2',2,1),(phi_2.T*M*phi_2).subs(params).evalf(3),

                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_1',2,1),sp.simplify((phi_2.T*M*phi_1)).subs(params).evalf(7),

                  sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('M', 2,2)*sp.MatrixSymbol('phi_2',2,1),(phi_1.T*M*phi_2).subs(params).evalf(7))

Eq(phi_1.T*M*phi_1, Matrix([[5.35e+3*newton*second**2/meter]]))

Eq(phi_2.T*M*phi_2, Matrix([[3.19e+3*newton*second**2/meter]]))

Eq(phi_2.T*M*phi_1, Matrix([[3.051758e-5*newton*second**2/meter]]))

Eq(phi_1.T*M*phi_2, Matrix([[3.051758e-5*newton*second**2/meter]]))

Es ist eine kleine numerische Differenz zu erkennen.

In [11]:
render.eq_display(sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_1',2,1),sp.simplify((phi_1.T*K*phi_1).subs(params)).evalf(3),
                  
                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_2',2,1),sp.simplify((phi_2.T*K*phi_2).subs(params)).evalf(3),

                  sp.MatrixSymbol('phi_2',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_1',2,1),sp.simplify((phi_2.T*K*phi_1).subs(params)).evalf(3),
                  
                  sp.MatrixSymbol('phi_1',2,1).T*sp.MatrixSymbol('K', 2,2)*sp.MatrixSymbol('phi_2',2,1),sp.simplify((phi_1.T*K*phi_2).subs(params)).evalf(3))

Eq(phi_1.T*K*phi_1, Matrix([[3.98e+6*newton/meter]]))

Eq(phi_2.T*K*phi_2, Matrix([[3.71e+6*newton/meter]]))

Eq(phi_2.T*K*phi_1, Matrix([[0]]))

Eq(phi_1.T*K*phi_2, Matrix([[0]]))

## Modale Analyse

Die Bewegungsgleichung für einen ungedämpften, periodisch, harmonisch angeregten Mehrmassenschwinger lässt sich folgend beschreiben:

$$\mathbf{M u''(t) + K u = F(t)}$$

Die Matrix-Gleichung beschreibt ein System aus Differentialgleichungen. Die Modale Analyse zielt darauf ab, diese zu entkoppeln. Bezogen auf den Mehrmassenschwinger heisst eine Entkoppelung, dass diese in Einmassenschwinger aufgeteilt werden. Dies wird nun schrittweise durchgeführt.

### Modal- und Spektralmatrix

Mittels der Modal- und Spektralmatrix können die generalisierten Grössen ermittelt werden. Diese sind die eigenschaften der einzelnen Einmassenschwinger. Die generalisierten Werte besitzen keine physikalischen Werte, sie sind abhängig von der Wahl der Eigenvektoren, welche bekanntlich von der Normierung abhängen.


Aufgrund der Bewegungsgleichung können die generalisierten Grössen bestimmt werden, es gilt:

$$\mathbf{\Phi^T M \Phi u''(t) + \Phi^T K \Phi u(t) = \Phi^T F(t)}$$

$$\mathbf{M^*u''(t) + K^* u(t) = F^* (t)}$$

Alle $N$-Eigenwerte und alle $N$-Eigenvektoren können kompakt
mit Matrizen ausgedrückt werden:


In [12]:
Phi = sp.Matrix([[phi_1, phi_2]])
Omega = sp.Matrix([[omega_1, 0],[0, omega_2]])

render.eq_display('Modalmatrix', 'Phi',
                  sp.MatrixSymbol('Phi', 2, 2), Phi.subs(params).evalf(4),
                  'Spektralmatrix', 'Omega^2',
                  sp.MatrixSymbol('Omega^2', 2, 2), (Omega**2).subs(params).evalf(4))

Eq(Modalmatrix, Phi)

Eq(Phi, Matrix([
[  1.0,    1.0],
[5.788, -3.455]]))

Eq(Spektralmatrix, Omega**2)

Eq(Omega^2, Matrix([
[744.5/second**2,                0],
[              0, 1160.0/second**2]]))

### Generalisierte Grössen


In [13]:
M_star = Phi.T * M * Phi
K_star = Phi.T * K * Phi
F_t_matrix = sp.Matrix([[F_t],[0]])
F_t_star = Phi.T * F_t_matrix

render.eq_display(sp.MatrixSymbol('M^\star', 2,2),sp.simplify(M_star.subs(params)).evalf(5),
                  sp.MatrixSymbol('K^\star', 2,2),sp.simplify(K_star.subs(params)).evalf(5),
                  sp.MatrixSymbol('F(t)', 2,1),sp.simplify(F_t_matrix.subs(params)).evalf(5),
                  sp.MatrixSymbol('F(t)^\star', 2,1),sp.simplify(F_t_star.subs(params)).evalf(5))

Eq(M^\star, Matrix([
[5350.6*newton*second**2/meter,                             0],
[                            0, 3193.8*newton*second**2/meter]]))

Eq(K^\star, Matrix([
[3.9837e+6*newton/meter,                      0],
[                     0, 3.7063e+6*newton/meter]]))

Eq(F(t), Matrix([
[800.0*newton*sin(12.6*t/second)],
[                              0]]))

Eq(F(t)^\star, Matrix([
[800.0*newton*sin(12.6*t/second)],
[800.0*newton*sin(12.6*t/second)]]))

### Kontrolle der modalen Transformation

In [14]:
omega_1_kontrolle = sp.sqrt(K_star[0] / M_star[0])
omega_2_kontrolle = sp.sqrt(K_star[3] / M_star[3])

render.eq_display('omega_1', omega_1.subs(params).simplify().evalf(3),
                  'omega_1_modal', omega_1_kontrolle.subs(params).simplify().evalf(3),
                  'omega_2', omega_2.subs(params).simplify().evalf(4),
                  'omega_2_modal', omega_2_kontrolle.subs(params).simplify().evalf(4))




Eq(omega_1, 27.3/second)

Eq(omega_1_modal, 27.3/second)

Eq(omega_2, 34.07/second)

Eq(omega_2_modal, 34.07/second)

## Stationäre Antwort

Die stationäre Antwort wird mittels des Vergrösserungsfaktors bestimmt.

Die entkoppelte Differentialgleichung ist nun die folgende:

$$m^\star_1 q_1''(t) + k^\star_1 q_1(t) = F^\star_1(t) = F^\star_1 \sin(\omega t)$$

$$m^\star_2 q_2''(t) + k^\star_2 q_2(t) = F^\star_2(t) = F^\star_2 \sin(\omega t)$$

Lösen lässt sich dies mit dem bekannten Ansatz:

$$q_n(t) = A_n \sin(\omega t) + B_n \cos(\omega)$$


### Verformung

In [15]:
zeta = sp.symbols('zeta')

V_1_omega = 1/(sp.sqrt((1-(omega/omega_1)**2)**2 + (2*zeta*(omega/omega_1))**2))

q_1_stat = F_0 / K_star[0]
q_1_max = q_1_stat*V_1_omega

render.eq_display('V_1(omega)', '1/(sqrt((1-(omega/omega_1)**2)**2 + (2*zeta_*(omega/omega_1))**2))',
                  'V_1(omega)', V_1_omega.subs(params).simplify().evalf(3),
                  'q_1_stat', unit.convert_to(q_1_stat.subs(params).simplify().evalf(3), unit.mm),
                  'q_1_max', unit.convert_to(q_1_max.subs(params).simplify().evalf(3), unit.mm),)

Eq(V_1(omega), 1/sqrt(4*omega**2*zeta_**2/omega_1**2 + (-omega**2/omega_1**2 + 1)**2))

Eq(V_1(omega), 1.27)

Eq(q_1_stat, 0.201*millimeter)

Eq(q_1_max, 0.255*millimeter)

In [16]:

V_2_omega = 1/(sp.sqrt((1-(omega/omega_2)**2)**2 + (2*zeta*(omega/omega_2))**2))

q_2_stat = F_0 / K_star[3]
q_2_max = q_2_stat*V_2_omega



render.eq_display('V_2(omega)', '1/(sqrt((1-(omega/omega_2)**2)**2 + (2*zeta_*(omega/omega_2))**2))',
                  'V_2(omega)', V_2_omega.subs(params).simplify().evalf(3),
                  'q_2_stat', unit.convert_to(q_2_stat.subs(params).simplify().evalf(3), unit.mm),
                  'q_2_max', unit.convert_to(q_2_max.subs(params).simplify().evalf(3), unit.mm))

Eq(V_2(omega), 1/sqrt(4*omega**2*zeta_**2/omega_2**2 + (-omega**2/omega_2**2 + 1)**2))

Eq(V_2(omega), 1.16)

Eq(q_2_stat, 0.216*millimeter)

Eq(q_2_max, 0.25*millimeter)

#### Effektive Deformation

In [17]:
u_1_stat = q_1_max*phi_1
u_2_stat = q_2_max*phi_2
u_stat = u_1_stat + u_2_stat

render.eq_display('Matrix([[u_1max],[u_2max]])',unit.convert_to(u_stat.subs(params).evalf(3),unit.mm))


Eq(Matrix([
[u_1max],
[u_2max]]), Matrix([
[0.505*millimeter],
[0.613*millimeter]]))

### Beschleunigung

In [18]:
V_a1_omega = omega**2 / omega_1**2 * V_1_omega

q_2_1_max =  F_0 / M_star[0] * V_a1_omega

render.eq_display('V_a1(omega)', V_a1_omega.subs(params).simplify().evalf(3),
                  'Derivative(q_max,t,2)', q_2_1_max.subs(params).simplify().evalf(3))

Eq(V_a1(omega), 0.271)

Eq(Derivative(q_max, (t, 2)), 0.0405*meter/second**2)

In [19]:
V_a2_omega = omega**2 / omega_2**2 * V_2_omega

q_2_2_max =  F_0 / M_star[3] * V_a2_omega

render.eq_display('V_a2(omega)', V_a2_omega.subs(params).simplify().evalf(3),'Derivative(q_max,t,2)', q_2_2_max.subs(params).simplify().evalf(3))

Eq(V_a2(omega), 0.158)

Eq(Derivative(q_max, (t, 2)), 0.0397*meter/second**2)

In [20]:
u_2_1_stat = q_2_1_max*phi_1
u_2_2_stat = q_2_2_max*phi_2
u_2_stat = u_2_1_stat + u_2_2_stat

render.eq_display('Matrix([[Derivative(u_1max,t,2)],[Derivative(u_2max,t,2)]])',u_2_stat.subs(params).evalf(3))


Eq(Matrix([
[Derivative(u_1max, (t, 2))],
[Derivative(u_2max, (t, 2))]]), Matrix([
[0.0802*meter/second**2],
[0.0975*meter/second**2]]))